In [ ]:
# Colab only: Run this cell to download/install packages
import sys
if 'google.colab' in sys.modules:
    !curl http://www.datasciencecourse.org/assignments/hw3_nlp.tar.gz --output hw3_nlp.tar.gz
    !tar -xzf hw3_nlp.tar.gz
    !mv hw3_nlp/* ./
    !pip install --upgrade --no-deps --force-reinstall git+https://github.com/locuslab/mugrade.git

# Natural Language Processing

**THIS IS ONLY FOR 15-688 STUDENTS**

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based on a TFIDF matrix and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essays written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution. If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers. They are a particularly interesting data set because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

In [1]:
import collections # we found collections.Counter and collections.defaultdict useful
import itertools
import gzip
import re
import numpy as np
import scipy.sparse as sp
import mugrade
from collections import Counter
from nltk.corpus import stopwords
from scipy.sparse import coo_matrix
from collections import defaultdict
from numpy import linalg
    

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18.  Specifically, the "pg18.txt.gz" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [2]:
def load_federalist_corpus(filename="pg18.txt.gz", encoding='utf8'):
    """ Load the federalist papers as a tokenized list of strings"""
    with gzip.open(filename, "rt", encoding=encoding) as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in sorted(punctuation, reverse=True):
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers


In [25]:
papers_content, authors, numbers=load_federalist_corpus()

You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `papers` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some punctuation tokens, which have been modified to include spaces both before and after the punctuation. The `authors` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally, the `numbers` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

### Q1: Bag of words, and TFIDF

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  **Note that you need to do this manually, and not via the scikit-learn Tfidf vectorizers you used in a previous assignment question**  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

You should create the tfidf vector using numpy matrix operations and not use an existing implementation.

Important: make sure you do _not_ include the empty token `""` as one of your terms. 

In [3]:
import math

In [4]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')


def tfidf(docs):
    
    # set constrain all words non repeated
    # list of documents 
    document_words = [doc.split() for doc in docs]
    
    # all unique words in all documents
    all_words = sorted(set(sum([doc.split() for doc in docs], [])))
    total_words = defaultdict(int)
    total_num_of_doc = len(docs)
    num_of_words = len(document_words)
    
    for unique_w in all_words:
        for i, doc in enumerate(document_words):
            if unique_w in doc:
                total_words[unique_w] += 1
                
    tfidf_row = []
    tfidf_col = []
    data=[]
    
    for i,doc in enumerate(document_words):
        for j,unique_w in enumerate(all_words):
            if unique_w in doc:
                score=(doc.count(unique_w))*np.log(total_num_of_doc/total_words[unique_w])
                if score != 0:
                    data.append(score)
                    tfidf_row.append(i)
                    tfidf_col.append(j)
    tfidf_matrix = coo_matrix((data, (tfidf_row, tfidf_col)), shape=(total_num_of_doc,len(all_words))).tocsr()

    return tfidf_matrix, all_words

Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>

### Q2: Cosine Similarity

Next, implement the following simple function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [11]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """
    tfidf=X.toarray()
    n=X.shape[0]
    Y=np.zeros((n,n))
    for i,doc1 in enumerate(tfidf):
        for j,doc2 in enumerate(tfidf):
            Y[i,j]=np.dot(doc1,doc2)/(linalg.norm(doc1)*linalg.norm(doc2))
    return Y


### Q3 Analyzing document authorship

Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Fill out the following function to compute and return these averages.

Hints:

1. fit a single TFIDF encoding to all papers and transform all papers using it before computing the similarity measure
2. for the cosine similarity to be useful when comparing documents, they must all be encoded the same way. Transform all papers together before comparing cosine similarity.
3. the unknown papers have author=`("HAMILTON","MADISON")`;

In [14]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def author_cosine_similarity(docs, authors):
    """
    Return a tuple of average cosine similarities between all the known papers for a given author
    and all the unknown papers.
    
    Args:
        docs: list of strings, where each string represents a space-separated
              document
        authors: list of lists, which each list contains the author (or potential authors) of a given document
    
    Returns: tuple: (hamilton_mcs, madison_mcs, jay_mcs)
        hamilton_mcs: Average cosine similarity between all the known Hamilton papers and all the unknown papers.
        madison_mcs: Average cosine similarity between all the known Madison papers and all the unknown papers.
        jay_mcs: Average cosine similarity between all the known Jay papers and all the unknown papers.
    """
    X,_ = tfidf(docs)
    cos_s = cosine_similarity(X)

    unknown_doc_pos = [pos for pos, author in enumerate(authors) if len(author) > 1]
    j_doc_pos = [pos for pos, author in enumerate(authors) if author[0].lower() == 'jay' and len(author)== 1]
    m_doc_pos = [pos for pos, author in enumerate(authors) if author[0].lower() == 'madison' and len(author)== 1]
    h_doc_pos = [pos for pos, author in enumerate(authors) if author[0].lower() == 'hamilton' and len(author)== 1]
    
            
    hamilton_mcs = cos_s[h_doc_pos,:][:,unknown_doc_pos].mean()
    madison_mcs = cos_s[m_doc_pos,:][:,unknown_doc_pos].mean()
    jay_mcs = cos_s[j_doc_pos,:][:,unknown_doc_pos].mean()
    
    
    return (hamilton_mcs, madison_mcs, jay_mcs)

Submitting tests for function author_cosine_similarity():
  Test 1 PASSED


Run the local test case, `author_cosine_similarity(*load_federalist_corpus()[:2])`, to see which author has the highest degree of similarity with the unknown essays.


## N-gram language model

### Q4 Building an n-gram language model

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class, and should do it in two parts:

### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You should do this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$-th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later functions, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen. We also build a `self.vocabulary` variable, which is just a `set` object containing all the unique words across the entire set of the input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  We'll make a small tweak to the formula for perplexity in the notes because we are skipping the first $n-1$ tokens; since we only compute (#words - $(n-1)$) terms in the sum, you should divide by this denominator and not just #words). Be careful to not multiply together probabilities that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

Specifically,

\begin{equation}
\mbox{Perplexity} = 2^{\frac{-\log_2 P\left(\mathrm{word}_1, \ldots, \mathrm{word}_N\right)}{N-(n+1)}} = 2^{\text{^}}\left(\frac{-\log_2 P(\mathrm{word}_1, \ldots, \mathrm{word}_N)}{N-(n-1)}\right)
\end{equation}

where

\begin{equation}
\log_2 P(\mathrm{word}_1, \ldots, \mathrm{word}_N) = \sum_{i=n}^N \log_2 P(\mathrm{word}_i | \mathrm{word}_{i-n+1}, \ldots, \mathrm{word}_{i-1})
\end{equation}

You'll want to be careful about vocabulary sizes when it comes to the Laplace smoothing term: make sure your vocabulary size $D$ is equal to the total number of unique words that occur in either the original data used to build the language model _or_ in the text we are evaluating the perplexity of (so the size of the union of the two).

In [113]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.
        
        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """
        
        self.counts = collections.defaultdict(dict) # Dict from space-separated "previous words" to a Dict of (next word, count). 
        self.count_sums = collections.defaultdict(int) # Dict from space-separated "previous words" to the total number of times they appear
        self.n = n
        aw=[]
        for doc in docs:
            all_words = doc.split()
            for j in range(0,len(all_words)-n+1):
                current_window_word = " ".join(all_words[j:j+(n-1)])
                adj_word = all_words[j+n-1] 
                if self.counts[current_window_word] == {}:
                    self.counts[current_window_word] = collections.defaultdict(int)

                if self.counts[current_window_word][adj_word] == 0:
                    self.counts[current_window_word][adj_word] = 1
                else:
                    self.counts[current_window_word][adj_word] += 1


                if self.count_sums[current_window_word] == 0:
                    self.count_sums[current_window_word] = 1
                else:
                    self.count_sums[current_window_word] += 1
                    
            aw+=doc.split()
        self.D=len(np.unique(aw))
        self.aw = aw
    
    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.
        
        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing
            
        Note: for the purposes of smoothing, the vocabulary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.
            
        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluated
                        under the model.
        """
        
        D = self.D
        n = self.n
        line = text.split()
        N = len(line)
        prob = []
        log2=0
        D = len((set(self.aw + line)))
        for i in range(n-1, len(line)):
            word = " ".join(line[i-n+1:i])
            if word in self.count_sums.keys():
                num=self.counts[word][line[i]]
                denom=self.count_sums[word]
            else:
                num=0
                denom=0
            prob_i = np.log((num+alpha)/(denom + (alpha*D)))
            log2+=prob_i
        perplexity = np.exp(-log2/(N-(n-1)))
        return perplexity


Running local tests for function LanguageModel():
  Test 1 PASSED
  Test 2 PASSED
  Test 3 PASSED
  Test 4 PASSED
  Test 5 PASSED
  Test 6 PASSED
  Test 7 PASSED
  Test 8 PASSED
  Test 9 PASSED


In [49]:
simpleLM = LanguageModel(["a a a a a", "a b a b"], 2)
# print(dict(simpleLM.counts))
print(dict(simpleLM.count_sums))

{}


### Q5 Author identification via language models 

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`). Fill in the following function to calculate and return the mean perplexities.

In [120]:
# @mugrade.local_tests
# @mugrade.submit_tests('28PY3HJpnZGQT8R2VgdC')

def mean_perplexity(docs, authors):
    """
    Evaluate the mean of the perplexity of the unknown Federalist papers for the language models
    from each of the three authors (again, using n=3 and alpha=1e-3)

    Args:
        docs: list of strings, where each string represents a space-separated document
        authors: list of lists, which each list contains the author (or potential authors) of a given document

    Returns: tuple: (perp_hamilton, perp_madison, perp_jay)
        perp_hamilton: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Hamilton
        perp_madison: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Madison
        perp_jay: floating point value, mean perplexity of the unknown Federalist papers for the language 
                       models from Jay
    """
    n = 3
    unknown_doc_pos = []
    j_doc_pos = []
    m_doc_pos = []
    h_doc_pos = []

    doc_w_j = []
    doc_w_m = []
    doc_w_h = []
    doc_w_u = []
    
    for pos, author in enumerate(authors):
        if len(author) > 1:
            unknown_doc_pos.append(pos)
            doc_w_u.append(docs[pos])
        else:
            if author[0].lower() == 'jay':
                j_doc_pos.append(pos)
                doc_w_j.append(docs[pos])
            elif author[0].lower() == 'madison':
                m_doc_pos.append(pos)
                doc_w_m.append(docs[pos])
            else:
                h_doc_pos.append(pos)
                doc_w_h.append(docs[pos])
                
    ham_model = LanguageModel(doc_w_h,n)
    mad_model = LanguageModel(doc_w_m,n)
    jay_model = LanguageModel(doc_w_j,n)
 
    perp_hamilton = sum([ham_model.perplexity(i) for i in doc_w_u])/len(doc_w_u)
    perp_madison = sum([mad_model.perplexity(i) for i in doc_w_u])/len(doc_w_u)
    perp_jay = sum([jay_model.perplexity(i) for i in doc_w_u])/len(doc_w_u)


    return (perp_hamilton, perp_madison, perp_jay)


Submitting tests for function mean_perplexity():
  Test 1 PASSED


Does the most likely author (i.e., the author with the smallest perplexity), match up with the author with the highest cosine similarity above?